In [1]:
# importing libraries
# импорт библиотек

import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import scipy.stats as ss
# import pingouin as pg
%matplotlib inline 

In [2]:
# reading dataframes
# чтение датафреймов

reg_data  = pd.read_csv('/Users/karimalibekov/Desktop/my_python/jupyter-k-alibekov/shared/', 
                        sep=';')
auth_data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-k-alibekov/shared/problem1-auth_data.csv', 
                        sep=';')
ab_data   = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-k-alibekov/shared/problem2.csv', 
                        sep=';')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/karimalibekov/Desktop/my_python/jupyter-k-alibekov/shared/'

## Overview & Preprocessing // Обзор и подготовка датафреймов

### DF №1 – reg_data // Датафрейм №1 – reg-data

In [ ]:
# overview
# просмотр

reg_data.head()

In [ ]:
# checking dtypes, non-null count and shape 
# смотрим на типы данных, пропущенные значения и размер датафрейма

reg_data.info()

In [ ]:
# checking missing values
# проверка упущенных значений

missing_values = reg_data.isnull().sum()

missing_values[missing_values > 0]

In [ ]:
# changing timestamp into datetime
# смена формата timestamp на datetime

reg_data['reg_ts'] = pd.to_datetime(reg_data['reg_ts'], unit='s')
reg_data.head()

In [ ]:
# renaming columns
# переименование колонок

reg_data = reg_data.rename(columns={'reg_ts':'reg_date', 'uid':'user_id'})
reg_data.head()

### DF №2 – auth_data // Датафрейм №2 – auth_data

In [ ]:
# overview
# просмотр

auth_data.head()

In [ ]:
# checking dtypes, non-null count and shape 
# смотрим на типы данных, пропущенные значения и размер датафрейма

auth_data.info()

In [ ]:
# checking missing values
# проверка упущенных значений

missing_values = auth_data.isnull().sum()

missing_values[missing_values > 0]

In [ ]:
# changing timestamp into datetime
# смена формата timestamp на datetime

auth_data['auth_ts'] = pd.to_datetime(auth_data['auth_ts'], unit='s')
auth_data.head()

In [ ]:
# renaming columns
# переименование колонок

auth_data = auth_data.rename(columns={'auth_ts':'auth_date', 'uid':'user_id'})
auth_data.head()

### DF №3 – ab_data // Датафрейм №3 – ab_data

In [ ]:
# overview
# просмотр

ab_data.head()

In [ ]:
# checking dtypes, non-null count and shape 
# смотрим на типы данных, пропущенные значения и размер датафрейма

ab_data.info()

In [ ]:
# checking missing values
# проверка упущенных значений

missing_values = ab_data.isnull().sum()

missing_values[missing_values > 0]

## Task 1 // Задание 1

## [ENG]

### Retention is one of the most important metrics in a company. Your task is to write a function that will count the retention of players (by days from the date of player registration) 

## [RUS]

### Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока).

In [ ]:
# merging dataframes
# слияние датафреймов

df = auth_data.merge(reg_data, on='user_id', how='left')
df.head()

In [ ]:
# checking dtypes and shape 
# смотрим на типы данных и размер датафрейма

df.info()

In [ ]:
# checking missing values
# проверка упущенных значений

missing_values = df.isnull().sum()

missing_values[missing_values > 0]

In [ ]:
# reordering columns
# перестановка колонок

df = df[['user_id', 'reg_date', 'auth_date']]
df

In [ ]:
# dropping minutes & seconds info from date
# удаляем информацию про минуты и секунды из даты

df['reg_date'] = df['reg_date'].apply(lambda x: datetime(x.year, x.month, x.day))
df['auth_date'] = df['auth_date'].apply(lambda x: datetime(x.year, x.month, x.day))

In [ ]:
# creating a year registration column 
# создание колонки с годом регистрации

df['reg_year'] = df.reg_date.dt.year
df.head()

In [ ]:
# visualization of number of registrations
# визуализация количества регистраций

fig = px.bar(df.groupby('reg_year', as_index=False).agg({'user_id':'count'}), 
             x='reg_year', y='user_id', labels={'reg_year':'year', 'user_id':'number of registrations'}, 
             title='Number of registrations over the years 1998–2020')
fig.show()

In [ ]:
# creating a sample of data, since the size of dataframe is big (for instance: data from 2020-01-01)
# берем сэмпл данных (за 2019 год, так как, так как размер датафрейма велик (например данные с 2020-01-01)

sample = df.query("reg_date >= '2020-01-01'")

In [ ]:
sample.head()

### Month Cohorts // Когорты по месяцам

### We assume that registration date == first authentification date == first interaction with game

### Мы предполагаем, что дата регистрации == дата первой аутентификации == первое взаимодействие с игрой

In [ ]:
# creating a cohort month with first day of the month
# создание когорт по первому дню месяца

sample['cohort_month'] = sample['reg_date'].apply(lambda x: dt.datetime(x.year, x.month, 1))

In [ ]:
# dropping reg_date, since it has the same meaning as cohort month due to assumption above
# отбрасываем reg_date, так как она имеет то же значение, что и cohort month, в связи с предположением выше

sample = sample[['user_id', 'auth_date', 'cohort_month']]
sample.head()

In [ ]:
# create a date element function to get a series for subtraction
# создание функции вычленения элемента даты в виде серии для последующего вычитания

def get_date_elements(df, column):
    day = df[column].dt.day
    month = df[column].dt.month
    year = df[column].dt.year
    return day, month, year 

In [ ]:
# get date elements for our cohort and invoice columns
# применение функции для колонок Cohort Month и Invoice Month

_,auth_month,auth_year =  get_date_elements(sample, 'auth_date')
_,cohort_month,cohort_year =  get_date_elements(sample, 'cohort_month')

In [ ]:
# check the series 
# проверка полученной серии

cohort_year[:10]

In [ ]:
# create a cohort index (in number of months, +1 is added so that there is no month zero)
# создание индекса когорты (в количестве месяцев, +1 добавляется, чтобы не было нулевого месяца)

year_diff = auth_year - cohort_year
month_diff = auth_month - cohort_month
sample['cohort_index'] = year_diff*12+month_diff+1
sample.tail(10)

In [ ]:
# count the user id using grouping by Cohort Month  and Cohort Index 
# считаем user id, используя группировку по месяцам когорты и индексу когорты

cohort_data = sample.groupby(['cohort_month','cohort_index'])['user_id'].apply(pd.Series.nunique).reset_index()
cohort_data

In [ ]:
# create a pivot table 
# создание сводной таблицы

cohort_table = cohort_data.pivot(index='cohort_month', columns=['cohort_index'],values='user_id')
cohort_table

In [ ]:
# change index format
# изменение формата индекса 

cohort_table.index = cohort_table.index.strftime('%B %Y')

In [ ]:
# visualization of our results in heatmap
# визуализация результата в виде тепловой карты

plt.figure(figsize=(20,8))
sns.heatmap(cohort_table, annot=True, cmap='magma');

In [ ]:
# cohort table as a percentage of the first month
# когортная таблица в виде процента от первого месяца

new_cohort_table = cohort_table.divide(cohort_table.iloc[:,0],axis=0)
new_cohort_table

In [ ]:
# heatmap visualization in %
# тепловая карта с %

plt.figure(figsize=(10,5))
sns.heatmap(new_cohort_table, annot=True, cmap='magma', fmt='.0%');

### Day Cohorts // Когорты по дням

In [ ]:
# creating another sample for day cohorts
# создаем еще один сэмпл данных для когорт по дням

sample_2 = df.query("reg_date >= '2020-01-01'")

In [ ]:
# choosing needed columns
# выбираем необходимые колонки

sample_2 = sample_2[['user_id', 'reg_date', 'auth_date']]
sample_2.head()

In [ ]:
# renaming for better understanding
# переименование для лучшего понимания

sample_2 = sample_2.rename(columns={'reg_date':'cohort_day'})
sample_2.head()

In [ ]:
# get date elements for our cohort and invoice columns
# применение функции для колонок Cohort Month и Invoice Month

auth_day,auth_month,_ =  get_date_elements(sample_2, 'auth_date')
cohort_day,cohort_month,_ =  get_date_elements(sample_2, 'cohort_day')


In [ ]:
# check the series 
# проверка полученной серии

cohort_day[:10]

In [ ]:
# create a cohort index (in number of days, +1 is added so that there is no day zero)
# создание индекса когорты (в количестве дней, +1 добавляется, чтобы не было нулевого дня)

day_diff = auth_day - cohort_day
month_diff = auth_month - cohort_month
sample_2['cohort_index'] = day_diff+1+month_diff*30
sample_2.tail()

In [ ]:
# count the user id using grouping by Cohort Month  and Cohort Index 
# считаем user id, используя группировку по месяцам когорты и индексу когорты

cohort_data_2 = sample_2.groupby(['cohort_day','cohort_index'])['user_id'].apply(pd.Series.nunique).reset_index()
cohort_data_2

In [ ]:
# create a pivot table 
# создание сводной таблицы

cohort_table_2 = cohort_data_2.pivot(index='cohort_day', columns=['cohort_index'],values='user_id')
cohort_table_2

In [ ]:
# change index format
# изменение формата индекса 

cohort_table_2.index = cohort_table_2.index.strftime('%d')

In [ ]:
# cohort table as a percentage of the first day
# когортная таблица в виде процента от первого дня

new_cohort_table_2 = cohort_table_2.divide(cohort_table_2.iloc[:,0],axis=0)
new_cohort_table_2

### Resulting Function // Итоговая функция 


In [ ]:
# creating a dataframe sample to put into function
# создаем еще один сэмпл датафрейма для подстановки в аргумент функции

sample_3 = df.query("reg_date >= '2020-09-01'")

In [ ]:
# function for cohort table
# функция, строящая когортную таблицу

def retention_analysis(df, user_id_column, min_date_column, action_date_column):
    
    data = df
    
    auth_day,auth_month,_ =  get_date_elements(data, action_date_column)
    cohort_day,cohort_month,_ =  get_date_elements(data, min_date_column)
    
    day_diff = auth_day - cohort_day
    month_diff = auth_month - cohort_month
    data['cohort_index'] = day_diff+1+month_diff*30
    
    cohort_data_ = data.groupby([min_date_column, 'cohort_index'])[user_id_column].apply(pd.Series.nunique).reset_index()

    cohort_table_ = cohort_data_.pivot(index=min_date_column, columns=['cohort_index'], values= user_id_column)
    
    cohort_table_.index = cohort_table_.index.strftime('%d')
    
    new_cohort_table_ = cohort_table_.divide(cohort_table_.iloc[:,0],axis=0)
    
    return new_cohort_table_

In [ ]:
# function call
# вызов функции

results = retention_analysis(sample_3, 'user_id', 'reg_date', 'auth_date')

results

In [ ]:
# heatmap visualization in %
# тепловая карта с %

plt.figure(figsize=(20,10))
sns.heatmap(results, annot=True, cmap='Blues', fmt='.0%', annot_kws={'fontsize':13});

## Task 2 // Задание 2

## ENG

### There are the results of an A/B test in which two groups of users were offered different sets of promotional offers. It is known that ARPU in the test group is 5% higher than in the control group. At the same time in the control group 1928 players out of 202103 turned out to be paying players, and in the test group - 1805 out of 202667.

### Which set of offers can be considered the best? What metrics should be analyzed to make the right decision and how?

## RUS

### Имеются результаты A/B теста, в котором двум группам пользователей предлагались различные наборы акционных предложений. Известно, что ARPU в тестовой группе выше на 5%, чем в контрольной. При этом в контрольной группе 1928 игроков из 202103 оказались платящими, а в тестовой – 1805 из 202667.

### Какой набор предложений можно считать лучшим? Какие метрики стоит проанализировать для принятия правильного решения и как?

## Plan // План: 

#### 1. All users with anomalies // Все пользователи с аномалиями 

    – descriptive statistics // описательная статистика

    – visualizations // визуализации

    – normality test // тест на нормальность

    – t-test // t-тест

    
#### 2. Paying users with anomalies // Платящие пользователи с аномалиями

    – descriptive statistics // описательная статистика

    – visualizations // визуализации

    – normality test // тест на нормальность

    – t-test // t-тест
    
#### 3. Paying users without anomalies // Платящие пользователи без аномалий

    – descriptive statistics // описательная статистика

    – visualizations // визуализации

    – normality test // тест на нормальность

    – t-test // t-тест
    
#### 4. Log version of paying users with anomalies // Платящие пользователи с аномалиями + логарифмическое преобразование

    – descriptive statistics // описательная статистика

    – visualizations // визуализации

    – normality test // тест на нормальность

    – t-test // t-тест

### 1. All users with anomalies (paying + not paying + anomalies) // Все пользователи с аномалиями

In [ ]:
ab_data.head()

In [ ]:
# size of dataframe
# размер датафрейма

ab_data.shape

In [ ]:
# unique users count
# количество уникальных пользователей

ab_data.user_id.nunique()

In [ ]:
# amount of users in each group
# количество пользователей в каждой группе

ab_data.testgroup.value_counts()

In [ ]:
# amount of users in each group in %
# количество пользователей в каждой группе в %

ab_data.testgroup.value_counts(normalize=True)

In [ ]:
# revenue sum, mean, median, max, min for each group
# сумма, медиана, среднее, максимальное и минимальное значение выручки для каждой группы 

ab_data.groupby('testgroup', as_index=False).agg({'user_id':'nunique', 'revenue':['sum', 'mean', 'median',
                                                                                  'max', 'min']})

**We see that mean is slightly bigger in group b, median is 0, since most of our users do not pay, also max value of 37433 in group a is very high, we must check if it is anomaly or not**
 
**Мы видим, что среднее значение немного больше в группе b, медиана равна 0, так как большинство наших пользователей не платят, также максимальное значение 37433 в группе a очень велико, мы должны проверить, является ли это аномалией или нет**

In [ ]:
# visualization of distribution of all a/b test participants (not paying + paying)
# визуализация распределения всех участников а/б тестирования (не платящие + платящие)

fig = px.histogram(ab_data, x="revenue", color="testgroup", color_discrete_sequence=["blue", "red"])
fig.show()

**It's hard to understand something from visualization above since we have big differences in values, so we need another way of displaying the data due to some filtering of origin dataframe**

**Из приведенной визуализации сложно что-то понять, так как у нас большие различия в значениях, поэтому нам нужен другой способ отображения данных, связанный с некоторой фильтрацией исходного датафрейма**

In [ ]:
# alternative way of above visualization
# альтернативный способ приведенной выше визуализации

sns.kdeplot(ab_data.query("testgroup == 'a'").revenue, color='red', fill=True)
sns.kdeplot(ab_data.query("testgroup == 'b'").revenue, color='blue', fill=True)

**Let's still check distributions for both groups for normality, these functions test the null hypothesis that a sample comes from a normal distribution.**

**Давайте все-таки проверим распределения для обеих групп на нормальность, эти функции проверяют нулевую гипотезу о том, что выборка имеет нормальное распределение.**

In [ ]:
print(ss.shapiro(ab_data.query("testgroup == 'a'").revenue))
print(ss.normaltest(ab_data.query("testgroup == 'a'").revenue))

In [ ]:
print(ss.shapiro(ab_data.query("testgroup == 'b'").revenue))
print(ss.normaltest(ab_data.query("testgroup == 'b'").revenue))

**In both groups we reject null hypotheses that data is distributed normally**

**В обеих группах мы отвергаем нулевые гипотезы о том, что данные распределены нормально**

#### This is a test for the null hypothesis that 2 independent samples have identical average (expected) values. This test assumes that the populations have identical variances by default.

In [ ]:
ss.ttest_ind(ab_data.query("testgroup == 'a'").revenue, ab_data.query("testgroup == 'b'").revenue)

### Conclusion // Выводы

    – We see that null hypothesis is not rejected in ttest, since p-value > 0.05
    – We need to compare groups data with filters and also to work with anomalies
    – So far we cannot say that a/b test is successful 

    – Мы видим, что нулевая гипотеза в ttest не отвергается, так как p-значение > 0,05
    – Нам необходимо сравнивать данные групп с помощью фильтров, а также работать с аномалиями
    – Пока мы не можем сказать, что a/b-тест является успешным 

### 2. Paying users with anomalies (only paying + anomalies) // Платящие пользователи с аномалиями

In [ ]:
# revenue sum, mean, median, max, min for each group among paying users
# сумма, медиана, среднее, максимальное и минимальное значение выручки для каждой группы среди платящих пользователей

ab_data.query("revenue > 0").groupby('testgroup', as_index=False).agg({'user_id':'nunique', 
                                                                       'revenue':['sum', 'mean', 'median', 
                                                                                  'max', 'min']})

**Median is not 0 anymore, since we cut all non-paying users, let's see if it helps to provide more statistical significant results**

**Медиана уже не равняется 0, так как мы отсекли всех неплатящих пользователей, посмотрим, поможет ли это получить более статистически значимые результаты**.

In [ ]:
# histogram of all paying users
# гистограмма всех платящих пользователей

fig = px.histogram(ab_data.query("revenue > 0"), x="revenue", color="testgroup", 
                   color_discrete_sequence=["red", "blue"])
fig.show()

**Here we can obviously see that we have ultra-high values in group A, which we can call anomalous, speaking from business point of view this segment is people who make biggest in-game purchases**

**Здесь мы видим, что в группе А мы имеем сверхвысокие значения, которые можно назвать аномальными, с точки зрения бизнеса этот сегмент - люди, совершающие самые крупные внутриигровые покупки**.

In [ ]:
# visualization of distribution of paying users by group with revenue mean
# визуализация распределения платящих пользователей по группам с отображением среднего значения выручки

plt.figure(figsize=(16, 8))

sns.kdeplot(ab_data.query("revenue > 0 & testgroup == 'a'").revenue, color='red', fill=True)
sns.kdeplot(ab_data.query("revenue > 0 & testgroup == 'b'").revenue, color='blue', fill=True)

plt.axvline(2663.998444, c='red', linestyle='--', label = 'mean (revenue > 0) for group a') 
plt.axvline(3003.658172, c='blue', linestyle='--', label = 'mean (revenue > 0) for group b')

plt.legend(bbox_to_anchor = (1.0, 1), loc = 'upper right') 

**In this plot we can see that in spite of group A has anomalous values, mean for group B is still more**

**На этом графике видно, что, несмотря на то, что группа А имеет аномальные значения, среднее значение для группы В все равно больше**.

In [ ]:
# visualization of distribution of paying users by group with revenue median
# визуализация распределения платящих пользователей по группам с отображением медианного значения выручки

plt.figure(figsize=(16, 8))

sns.kdeplot(ab_data.query("revenue > 0 & testgroup == 'a'").revenue, color='red', fill=True)
sns.kdeplot(ab_data.query("revenue > 0 & testgroup == 'b'").revenue, color='blue', fill=True)

plt.axvline(311, c='red', linestyle='--', label = 'median (revenue > 0) for group a') 
plt.axvline(3022, c='blue', linestyle='--', label = 'median (revenue > 0) for group b')

plt.legend(bbox_to_anchor = (1.0, 1), loc = 'upper right') 

**Median for group B is also bigger**

**Медиана для группы B также больше**

In [ ]:
# violin plot to see quantiles, anomalies among paying users
# скрипичный график для отображения квантилей, поиска аномалий среди платящих пользователей

fig = px.violin(ab_data.query("revenue > 0"), y=ab_data.query("revenue > 0").revenue, x="testgroup", color="testgroup", 
                color_discrete_sequence=["red", "blue"], box=True, points="all")
fig.show()

In [ ]:
# share of anomalous paying users among all paying users
# доля аномальных платящих пользователей среди всех платящих пользователей

len(ab_data.query("revenue > 20000")) / len(ab_data.query("revenue > 0"))

**Share of anomalous users is less than 5%, so we can drop it in future research**

**Доля аномальных пользователей составляет менее 5%, поэтому мы можем отказаться от нее в будущих исследованиях**.

In [ ]:
print(ss.shapiro(ab_data.query("revenue > 0 & testgroup == 'a'").revenue))
print(ss.normaltest(ab_data.query("revenue > 0 & testgroup == 'a'").revenue))

In [ ]:
print(ss.shapiro(ab_data.query("revenue > 0 & testgroup == 'b'").revenue))
print(ss.normaltest(ab_data.query("revenue > 0 & testgroup == 'b'").revenue))

**In both groups we reject null hypotheses that data is distributed normally**

**В обеих группах мы отвергаем нулевые гипотезы о том, что данные распределены нормально**

In [ ]:
ss.ttest_ind((ab_data.query("revenue > 0 & testgroup == 'a'").revenue), 
             ab_data.query("revenue > 0 & testgroup == 'b'").revenue)

### Conclusion // Выводы

    – We see that null hypothesis is not rejected in ttest, since p-value > 0.05
    – We need to compare groups data without anomalies
    – So far we cannot say that a/b test is successful 

    – Мы видим, что нулевая гипотеза в ttest не отвергается, так как p-значение > 0,05
    – Нам необходимо сравнивать данные групп без аномалий
    – Пока мы не можем сказать, что a/b-тест является успешным 

### 3. Paying users without anomalies (paying + without anomalies) // Платящие пользователи без аномалий

In [ ]:
ab_data_2 = ab_data.query("0 < revenue < 20000")
ab_data_2

**Rule of revenue < 20000 helps us to inspect data without anomalous values**

**Правило выручки < 20000 помогает нам проверять данные без аномальных значений**

In [ ]:
ab_data_2.describe()

**Now we see that max value of revenue for both groups is 4000**

**Теперь мы видим, что максимальное значение выручки для обеих групп равно 4000**

In [ ]:
ab_data_2.groupby('testgroup', as_index=False).agg({'user_id':'nunique', 
                                                    'revenue':['sum', 'mean', 'median', 
                                                                        'max', 'min']})

In [ ]:
plt.figure(figsize=(16, 8))

sns.kdeplot(ab_data_2.query("testgroup == 'a'").revenue, color='red', fill=True)
sns.kdeplot(ab_data_2.query("testgroup == 'b'").revenue, color='blue', fill=True)

plt.axvline(302.458172, c='red', linestyle='--', label = 'mean (20000 > revenue > 0) for group a') 
plt.axvline(3003.658172, c='blue', linestyle='--', label = 'mean (20000 > revenue > 0 for group b')

plt.legend(bbox_to_anchor = (1.0, 1), loc = 'upper right') 

In [ ]:
plt.figure(figsize=(16, 8))

sns.kdeplot(ab_data_2.query("testgroup == 'a'").revenue, color='red', fill=True)
sns.kdeplot(ab_data_2.query("testgroup == 'b'").revenue, color='blue', fill=True)

plt.axvline(305, c='red', linestyle='--', label = 'median (20000 > revenue > 0) for group a') 
plt.axvline(3022, c='blue', linestyle='--', label = 'median (20000 > revenue > 0 for group b')

plt.legend(bbox_to_anchor = (1.0, 1), loc = 'upper right') 

**Now without anomalous values we see that group B has significantly higher values in mean and median values**

**Теперь без аномальных значений мы видим, что группа В имеет значительно более высокие значения средних и медианных величин**

In [ ]:
fig = px.violin(ab_data_2, y="revenue", x="testgroup", color="testgroup", color_discrete_sequence=["red", "blue"], 
                box=True, points="all")
fig.show()

**Violin plot also shows us that we don't have 'tail' like before in group A**

**Скрипичный график также показывает, что у нас нет "хвоста", как раньше в группе A**

**Let's see distributions for both groups and make normality tests + ttest in the end**

**Посмотрим распределения для обеих групп и проведем проверку на нормальность + ttest в конце**

In [ ]:
sns.kdeplot(ab_data_2.query("testgroup == 'a'").revenue, color='red', fill=True)

In [ ]:
print(ss.shapiro(ab_data_2.query("testgroup == 'a'").revenue)) 
print(ss.normaltest(ab_data_2.query("testgroup == 'a'").revenue))

In [ ]:
sns.kdeplot(ab_data_2.query("testgroup == 'b'").revenue, color='blue', fill=True)

In [ ]:
print(ss.shapiro(ab_data_2.query("testgroup == 'b'").revenue))
print(ss.normaltest(ab_data_2.query("testgroup == 'b'").revenue))

In [ ]:
ss.ttest_ind((ab_data_2.query("testgroup == 'a'").revenue), (ab_data_2.query("testgroup == 'b'").revenue))

### Conclusion // Выводы

    – Our distributions are still not normal, but we have enough data to perform a test without loosing accuracy of p-value 
    – We see that null hypothesis is rejected in ttest, since p-value < 0.05
    – We can say that a/b test is successful (statistically significant + test group shows better results)
    – We also can compare paying users with anomalies with log-transformation
    
    - Наши распределения все еще не являются нормальными, но у нас достаточно данных для проведения теста без потери точности p-значения 
    - Мы видим, что нулевая гипотеза в ttest отвергается, поскольку p-value < 0,05.
    - Можно сказать, что a/b-тест прошел успешно (статистически значимо + тестовая группа показала лучшие результаты)
    – Мы также можем сравнить платящих пользователей с аномалиями с помощью логарифмического преобразования

### 4. Log version of paying users with anomalies (only paying + anomalies) // Платящие пользователи с аномалиями + логарифмическое преобразование

In [ ]:
ab_data.query("revenue > 0")

**Visualizations of distributions before log-transformation**

**Визуализация распределений до лог-преобразования**

In [ ]:
sns.kdeplot(ab_data.query("revenue > 0 & testgroup == 'a'").revenue, color='red', fill=True)

In [ ]:
sns.kdeplot(ab_data.query("revenue > 0 & testgroup == 'b'").revenue, color='blue', fill=True)

**Log-transformation**

**Лог-преобразование**

In [ ]:
revenue_a_log = np.log(ab_data.query("revenue > 0 & testgroup == 'a'").revenue)
revenue_a_log

In [ ]:
revenue_b_log = np.log(ab_data.query("revenue > 0 & testgroup == 'b'").revenue)
revenue_b_log

**Visualizations of distributions after log-transformation**

**Визуализация распределений после лог-преобразования**

In [ ]:
sns.kdeplot(revenue_a_log, color='red', fill=True)

In [ ]:
sns.kdeplot(revenue_b_log, color='blue', fill=True)

In [ ]:
print('revenue_a_log mean is: ', revenue_a_log.mean())
print('revenue_b_log mean is: ', revenue_b_log.mean())
print('revenue_a_log median is: ', revenue_a_log.median())
print('revenue_b_log median is: ', revenue_b_log.median())

**After log-transformation group B is still leader in revenue mean and median values**

**После лог-трансформации группа B по-прежнему лидирует по средним и медианным значениям выручки**

In [ ]:
print(ss.shapiro(revenue_a_log))
print(ss.normaltest(revenue_a_log))

In [ ]:
print(ss.shapiro(revenue_b_log))
print(ss.normaltest(revenue_b_log))

In [ ]:
ss.ttest_ind(revenue_a_log, revenue_b_log)

### Conclusion // Выводы

    – Our distributions are still not normal, but we have enough data to perform a test without loosing accuracy of p-value 
    – We see that null hypothesis is rejected in ttest, since p-value < 0.05
    – We can say that a/b test is successful (statistically significant + test group shows better results)
    – Anomalies do not affect signigicance test with log-transformation of data
    
    - Наши распределения все еще не являются нормальными, но у нас достаточно данных для проведения теста без потери точности p-значения 
    - Мы видим, что нулевая гипотеза в ttest отвергается, поскольку p-value < 0,05.
    - Можно сказать, что a/b-тест прошел успешно (статистически значимо + тестовая группа показала лучшие результаты)
    - Аномалии не влияют на проверку значимости при логарифмическом преобразовании данных

## Results // Итоги по заданию

    - In any of the scenarios, group B shows better values compared to group A (paying / non-paying / with anomalies / without anomalies / log-transformation)
    - Considering still that ARPU in the test group is 5% higher than in the control group, we can clearly conclude that the test group has better results
    - The company should use the set of suggestions for the test group for all groups

    – В любом из сценариев группа Б показывает лучшие значения по сравнению с группой А (платящие / не платящие / с аномалиями / без аномалий / лог-трансформация)
    – Учитывая еще, что ARPU в тестовой группе выше на 5%, чем в контрольной, можно сделать однозначный вывод, что тестовая группа имеет лучшие результаты
    – Компании стоит использовать набор предложений для тестовой группы для всех групп

## Task 3 // Задание 3

## ENG

### The Plants & Gardens game features themed events every month for a limited time. In them, players can get unique items for the garden and characters, additional coins or bonuses. The rewards require completing a series of levels in a certain amount of time. What metrics can be used to evaluate the results of the most recent past event?

### Suppose in another event we have complicated the event mechanics so that every time the player fails to complete a level, the player will be rolled back several levels. Would the set of outcome evaluation metrics change? If so, how?

## RUS

### В игре Plants & Gardens каждый месяц проводятся тематические события, ограниченные по времени. В них игроки могут получить уникальные предметы для сада и персонажей, дополнительные монеты или бонусы. Для получения награды требуется пройти ряд уровней за определенное время. С помощью каких метрик можно оценить результаты последнего прошедшего события?

### Предположим, в другом событии мы усложнили механику событий так, что при каждой неудачной попытке выполнения уровня игрок будет откатываться на несколько уровней назад. Изменится ли набор метрик оценки результата? Если да, то как?

–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
### Metrics for scenario 1 (each unsuccessful level attempt will NOT cause the player to roll back a few levels)

> Average number of levels completed by the user per event

> Average time to complete each level per event

> Average time to complete all levels per event (for those who have completed all levels)

> Churn rate for each level (which levels were particularly hard for users)

> Average number of unique items a user received by different categories (garden, characters, extra coins, bonuses)

> The number of new unique users during the event

> Rate of users returning to the app during the themed event

> Average amount of money spent by the user during the themed event period

> Average amount of money spent by the user during the themed event period relative to the previous month + next month (reflexive metric)

> Average amount of time spent in the app

### Additional metrics (metrics for scenario 1 are also relevant) for scenario 2 (for each unsuccessful attempt to complete a level, the player SHOULD roll back a few levels)

> Average number of rollbacks per user 

> Average number of rollbacks per user by level

> Metric showing how many times the levels had to be retried due to a rollback (summing up the levels retried after the rollback - total number of levels)

### Метрики для сценария №1 (при каждой неудачной попытке выполнения уровня игрок НЕ БУДЕТ откатываться на несколько уровней назад)

> Среднее количество пройденных уровней пользователем за событие

> Среднее время прохождения каждого уровня за событие

> Среднее время прохождения всех уровней за событие (для тех, кто прошел все уровни)

> Показатель оттока для каждого уровня (какие уровни были особенно тяжелы для пользователей)

> Среднее количество полученных уникальных предметов пользователем по разным категориям (сад, персонажи, доп. монеты, бонусы)

> Количество новых уникальных пользователей во время проведения события

> Показатель возврата пользователей в приложение на время тематического события

> Среднее количество потраченных денег пользователем в период тематического события

> Среднее количество потраченных денег пользователем в период тематического события относительно предыдущего месяца + следующего месяца (рефлексивная метрика)

> Среднее количество проведенного времени в приложении

### Дополнительные метрики (метрики для сценария №1 также релевантны) для сценария №2 (при каждой неудачной попытке выполнения уровня игрок БУДЕТ откатываться на несколько уровней назад)

> Среднее количество откатов на пользователя 

> Среднее количество откатов на пользователя по уровням

> Метрика, показывающая сколько раз пришлось дополнительно пройти уровни из-за отката (суммируются заново пройденные уровни после отката - общее количество уровней)